In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pyspark

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__} pyspark-shell'
os.environ['SPARK_SUBMIT_OPTS'] = '-Djdk.security.auth.login.Config=ignore'

In [5]:
KAFKA_BROKER_URL = "localhost:9092"
KAFKA_TOPIC = "ev_topic"

In [6]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sseclient

Note: you may need to restart the kernel to use updated packages.


In [8]:
import csv
import json
import time
from kafka import KafkaProducer

KAFKA_BROKER_URL = "localhost:9092"
KAFKA_TOPIC = "ev_topic"

producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER_URL,
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

file_path = '../input/Electric_Vehicle_Population_Data.csv'
print(f"Streaming data to {KAFKA_TOPIC}...")

with open(file_path, mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        producer.send(KAFKA_TOPIC, value=row)

producer.flush()

Streaming data to ev_topic...


In [9]:
from pyspark.sql import SparkSession
import pyspark

# Initialize the Spark Session with Kafka support
spark = SparkSession.builder \
    .appName("EV-Streaming-Exercise") \
    .config("spark.jars.packages", f"org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__}") \
    .config("spark.sql.streaming.checkpointLocation", "./checkpoint") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/06 16:09:32 WARN Utils: Your hostname, DESKTOP-AMKTFU4, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/06 16:09:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/lihicohen122/Pyspark/.venv_new/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/lihicohen122/.ivy2.5.2/cache
The jars for the packages stored in: /home/lihicohen122/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0c83b49-b5c2-46a3-a2df-8af6c8a5602f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.1.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.1.0 in central
	found org.apache.kafka#kafka-clien

In [10]:
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import col, from_json

# 1. Define the schema
schema = StructType() \
    .add("City", StringType()) \
    .add("Model Year", StringType())

# 2. Connect to the Kafka stream
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
    .option("subscribe", KAFKA_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

# 3. Transform the raw Kafka 'value' into columns
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

In [11]:
from pyspark.sql.functions import desc
# Find top 3 cities for 2023
top_cities_2023 = parsed_df.filter(col("Model Year") == "2023") \
    .groupBy("City") \
    .count() \
    .orderBy(desc("count")) \
    .limit(3)

# Start the stream
query = top_cities_2023.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(availableNow=True) \
    .start()

query.awaitTermination()

26/01/06 16:10:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
26/01/06 16:10:04 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+--------+-----+
|    City|count|
+--------+-----+
| Seattle|22708|
| Tukwila|10517|
|Bellevue| 9706|
+--------+-----+

